 ###  CTF Challenge: Visual Vortex - Decoding the Anomaly







**Dataset**: [U.S. International Air Traffic Statistics (1990–2023)](https://www.transtats.bts.gov/)  
**Flag Format**: `CTF{Country_Year_Event}`  

 ## 📥 Setup & Data Loading (4 points)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans

a = pd.read_csv(r"D:\Documents 2\VScode projects\ROBAI2\challenges\data\air_traffic.csv")

# dataset comprehension  (11 point)

### read and analyse the data then respond to these questions :

### Core Questions (must answer atleast 3 of these!! ) 

1. Which country has the highest total passenger traffic (1990–2023)?  
2. How is passenger traffic distributed across countries?  
3. Which year had the highest and lowest year-over-year (YoY) passenger growth?  
4. What is the average monthly passenger volume across all years?  
5. How is air freight volume distributed globally?  

### Bonus Questions  

1. What is the average number of passengers per flight for each country?  
2. Which country has the highest number of recorded flights?  
3. Which month historically has the highest air traffic volume?  

### Advanced Bonus Questions  

1. Which country shows the strongest correlation between passenger traffic and freight volume?  
2. Are there seasonal patterns in air traffic for specific countries?  
3. Which country recovered fastest to pre-COVID (2019) passenger levels post-2020?  

In [ ]:
# 1. country with the highest total passenger traffic (1990–2023)
total_passengers_by_country = a.groupby('Country')['Passengers'].sum()
highest_passenger_country = total_passengers_by_country.idxmax()
highest_passenger_traffic = total_passengers_by_country.max()
print(f"Country with highest total passenger traffic: {highest_passenger_country} ({highest_passenger_traffic})")

# 2. passenger traffic distribution across countries
plt.figure(figsize=(12, 6))
sns.barplot(x=total_passengers_by_country.index, y=total_passengers_by_country.values)
plt.title("passenger traffic distribution across countries (1990–2023)")
plt.ylabel("Total Passengers")
plt.xlabel("Country")
plt.xticks(rotation=45)
plt.show()

# 3. year witht the highest and lowest year-over-year (YoY) passenger growth
a['Total_Passengers_Per_Year'] = a.groupby('Year')['Passengers'].transform('sum')
a['YoY_Growth'] = a['Total_Passengers_Per_Year'].pct_change() * 100
highest_growth_year = a.loc[a['YoY_Growth'].idxmax(), 'Year']
lowest_growth_year = a.loc[a['YoY_Growth'].idxmin(), 'Year']
print(f"Year with highest YoY passenger growth: {highest_growth_year}")
print(f"Year with lowest YoY passenger growth: {lowest_growth_year}")

# 4. average amount of passengers for each month across all years
average_passengers_per_month = a.groupby('Month')['Passengers'].mean()
print("Average passengers per month:")
print(average_passengers_per_month)

# 5. air freight volume distribution globally
total_freight_by_country = a.groupby('Country')['Freight (tonnes)'].sum()
plt.figure(figsize=(12, 6))
sns.barplot(x=total_freight_by_country.index, y=total_freight_by_country.values)
plt.title("air freight volume distribution globally (1990–2023)")
plt.ylabel("Total Freight (tonnes)")
plt.xlabel("Country")
plt.xticks(rotation=45)
plt.show()

#Bonus question
#average number of passengers per flight for each country
average_passengers_per_flight = a.groupby('Country')['Passengers'].mean()
print("Average number of passengers per flight for each country:")
print(average_passengers_per_flight)

 ## 🧩 Part 1: Data Inspection (5 points)
 **Q1** : How many missing values exist in the `Passengers` and `Flights` columns?  
 **Q2** : What is the time range of the dataset?

In [ ]:
missing_values_flights = a['Flights'].isnull().sum()
missing_values_passengers = a['Passengers'].isnull().sum()
print(f"Missing values in flights: {missing_values_flights}")
print(f"Missing values in passengers: {missing_values_passengers}")

time_range = a['Year'].unique()
time_range.sort()
print(f"Time range of the dataset: {time_range[0]} to {time_range[-1]}")


## 🛠️ Part 2: Data Augmentation  (10 points)
 **Q3**: Clean the dataset. Justify your strategy.  
 **Q4**: Create `Passengers_per_Flight` and `YoY_Growth`. What do they measure?

In [ ]:
a = a.dropna()
a = a.drop_duplicates()
a = a.reset_index(drop=True)
a = a.rename(columns={'Freight (tonnes)': 'Freight'})

# Fixing country names to ensure consistency
a['Country'] = a['Country'].replace({
    'Chinaa': 'China',
    'Chna': 'China',
    'Mexiko': 'Mexico',
    'Germnay': 'Germany',
    'Deutschland': 'Germany',
    'Itly': 'Italy',
    'United States': 'USA',
    'Mehico': 'Mexico',
    'Itlay': 'Italy'
})

# Dropping rows with invalid or inconsistent data
a = a[a['Passengers'] > 0]  
a = a[a['Flights'] > 0]     
a = a[a['Freight'] > 0]     

# Remove rows with invalid months (13 or 0)
a = a[(a['Month'] >= 1) & (a['Month'] <= 12)]

# Reset the index after removing invalid rows
a = a.reset_index(drop=True)

a = a.reset_index(drop=True)


## 📈 Part 3: Time Series Visualization (10 point)
 **Q5**: Plot passenger trends for a country. What anomalies exist?  
 **Q6**: Add a rolling average. What long-term trend emerges?

In [ ]:
# Plot passenger trends for a specific country (e.g., 'USA')
usa_data = a[a['Country'] == 'USA']
plt.figure(figsize=(12, 6))
plt.plot(usa_data['Year'], usa_data['Passengers'], label='Passenger Traffic', marker='o')
plt.title('Passenger Trends for USA (1990–2023)')
plt.xlabel('Year')
plt.ylabel('Passengers')
plt.legend()
plt.grid(True)
plt.show()
print("september happened lol! and apparently americans didn't care for the pandemic") 

# Add a rolling average to smooth the trend
usa_data = a[a['Country'] == 'USA'].copy()
usa_data['Rolling_Avg'] = usa_data['Passengers'].rolling(window=3, center=True).mean()
plt.figure(figsize=(12, 6))
plt.plot(usa_data['Year'], usa_data['Passengers'], label='Passenger Traffic', marker='o', alpha=0.5)
plt.plot(usa_data['Year'], usa_data['Rolling_Avg'], label='3-Year Rolling Average', color='red', linewidth=2)
plt.title('Passenger Trends for USA with Rolling Average (1990–2023)')
plt.xlabel('Year')
plt.ylabel('Passengers')
plt.legend()
plt.grid(True)
plt.show()

# Plot passenger trends for all countries in one figure
plt.figure(figsize=(15, 8))
for country in a['Country'].unique():
    country_data = a[a['Country'] == country]
    plt.plot(country_data['Year'], country_data['Passengers'], label=country, marker='o', alpha=0.7)

plt.title('Passenger Trends for All Countries (1990–2023)')
plt.xlabel('Year')
plt.ylabel('Passengers')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(True)
plt.tight_layout()
plt.show()

# Generate rolling average for all countries and plot them
plt.figure(figsize=(15, 8))
for country in a['Country'].unique():
    country_data = a[a['Country'] == country].copy()
    country_data['Rolling_Avg'] = country_data['Passengers'].rolling(window=3, center=True).mean()
    plt.plot(country_data['Year'], country_data['Rolling_Avg'], label=f'{country} (Rolling Avg)', alpha=0.7)

plt.title('Passenger Trends with Rolling Average for All Countries (1990–2023)')
plt.xlabel('Year')
plt.ylabel('Passengers (Rolling Average)')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(True)
plt.tight_layout()
plt.show()

print("i guess what we could make out of these is that september 11th was a big deal for the us, and the pandemic was a big deal for everyone but the us")

## 🌍 Part 4: Geospatial Visualization (recommended) (10 points)
 **Q7**: Map freight volume by country. Which regions are outliers?  
 **Q8**: Did freight to China drop in 2022?

In [ ]:
# Q7: Map freight volume by country and identify outliers
# Aggregate freight volume by country
total_freight_by_country = a.groupby('Country')['Freight'].sum().reset_index()

# Sort countries by freight volume for better visualization
total_freight_by_country = total_freight_by_country.sort_values(by='Freight', ascending=False)

# Plot freight volume by country
plt.figure(figsize=(15, 8))
plt.bar(total_freight_by_country['Country'], total_freight_by_country['Freight'], color='skyblue')
plt.title('Freight Volume by Country (1990–2023)', fontsize=16)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Total Freight (tonnes)', fontsize=12)
plt.xticks(rotation=90, fontsize=10)
plt.tight_layout()
plt.show()

# Identify outliers using the IQR method
Q1 = total_freight_by_country['Freight'].quantile(0.25)
Q3 = total_freight_by_country['Freight'].quantile(0.75)
IQR = Q3 - Q1
outliers = total_freight_by_country[total_freight_by_country['Freight'] > Q3 + 1.5 * IQR]
print("Outlier countries with exceptionally high freight volume:")
print(outliers)

# Q8: Check if freight to China dropped in 2022
china_freight_2022 = a[(a['Country'] == 'China') & (a['Year'] == 2022)]['Freight'].sum()
china_freight_2021 = a[(a['Country'] == 'China') & (a['Year'] == 2021)]['Freight'].sum()

if china_freight_2022 < china_freight_2021:
    print(f"Freight to China dropped in 2022. 2021: {china_freight_2021}, 2022: {china_freight_2022}")
else:
    print(f"Freight to China did not drop in 2022. 2021: {china_freight_2021}, 2022: {china_freight_2022}")

## 🔗 Part 5: Correlation Analysis (10 points)
 **Q9**: Are passengers, flights, and freight strongly correlated?  
 **Q10**: Do clusters emerge in the 3D plot?

In [ ]:
# Q9: Correlation analysis between Passengers, Flights, and Freight
correlation_matrix = a[['Passengers', 'Flights', 'Freight']].corr()
print("Correlation matrix:")
print(correlation_matrix)

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation between Passengers, Flights, and Freight')
plt.show()

# Q10: 3D clustering plot for Passengers, Flights, and Freight
# Perform clustering (e.g., 3 clusters)
kmeans = KMeans(n_clusters=3, random_state=42)
a['Cluster'] = kmeans.fit_predict(a[['Passengers', 'Flights', 'Freight']].dropna())

# 3D scatter plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(
    a['Passengers'], a['Flights'], a['Freight'], c=a['Cluster'], cmap='viridis', alpha=0.7
)
ax.set_xlabel('Passengers')
ax.set_ylabel('Flights')
ax.set_zlabel('Freight')
plt.title('3D Clustering of Passengers, Flights, and Freight')
plt.colorbar(scatter, label='Cluster')
plt.show()

## 🚨 Part 6: Anomaly Detection (15 points)
 **Q11**: What’s the worst YoY passenger drop?  
 **Q12**: What event caused it?

In [ ]:
# Q11: Identify the worst YoY passenger drop
a['Total_Passengers_Per_Year'] = a.groupby('Year')['Passengers'].transform('sum')
a['YoY_Growth'] = a['Total_Passengers_Per_Year'].pct_change() * 100
worst_yoy_drop = a['YoY_Growth'].min()
worse_yoy_year = a.loc[a['YoY_Growth'].idxmin(), 'Year']
print(f"Worst YoY passenger drop: {worst_yoy_drop:.2f}% in {worse_yoy_year}")

# Q12: Analyze the event causing the worst drop
if worse_yoy_year == 2020:
    print("The worst YoY drop occurred in 2020, likely due to the COVID-19 pandemic, which severely impacted global air travel.")
else:
    print(f"The worst YoY drop occurred in {worse_yoy_year}. Further investigation is needed to determine the cause.")

## 🏁 Part 7: Flag Submission (30 point)
**Q13**: Submit your flag as `CTF{Country_Year_Event}`.

In [ ]:
# Define the event
if worse_yoy_year == 2020:
    event = "COVID-19"
    country = "Global"  # The pandemic affected all countries
else:
    event = "Unknown"
    country = "Unknown"

# Analyze passenger traffic for the USA in 2001
usa_2001 = a[(a['Country'] == 'USA') & (a['Year'] == 2001)]
total_passengers_2001 = usa_2001['Passengers'].sum()

# Compare with 2000 to calculate the drop
usa_2000 = a[(a['Country'] == 'USA') & (a['Year'] == 2000)]
total_passengers_2000 = usa_2000['Passengers'].sum()

passenger_drop = total_passengers_2000 - total_passengers_2001
drop_percentage = (passenger_drop / total_passengers_2000) * 100

# Flag for the global impact of COVID-19 in 2020
flag_covid = "CTF{Global_2020_COVID-19}"
print(f"Flag 1: {flag_covid}")

# Flag for the impact of 9/11 in the USA in 2001
flag_911 = "CTF{USA_2001_9/11}"
print(f"Flag 2: {flag_911}")
a.to_csv(r"D:\Documents 2\VScode projects\ROBAI2\challenges\data\air_traffic_cleaned.csv", index=False)


 ## 🎯 Bonus Points (Advanced Techniques)
 **Extra credit will be awarded for implementing the following methods:**  

        * **Dimensionality Reduction (PCA)**

        * **Time Series Decomposition**

        * **Clustering (K-Means/DBSCAN)**  

        
 ---
 **Scoring Boost**:  
 - **+15%**: For implementing PCA + Clustering  
 - **+10%**: For time series decomposition or automated anomaly detection
 - **+20%**: for  creative / original ideas or implimentations  (justifications and context  will also be considered )
 